In [68]:
from copy import deepcopy
import heapq
import time
import hashlib
import pyttsx3
engine = pyttsx3.init()


class Agente:
    def __init__(self):
        self.percepciones = None
        self.acciones = None
        self.habilitado = True
    
    def programa(self):
        raise Exception("implementar")

class Entorno:
  def __init__(self):
    self.agentes =[]
  
  def get_percepciones (self,a):
    raise Exception("implementar")
  
  def ejecutar (self, a):
    raise Exception("Implementar")
  
  def finalizado (self):
    return not any (a.habilitado for a in self.agentes)
  
  def run(self):
    while not self.finalizado():
      for a in self.agentes:
        print(a, " ")
        self.get_percepciones(a)
        self.ejecutar(a)
  
  def insertar (self,a):
    self.agentes.append(a)


class AgenteBuscador(Agente):
    def __init__(self):
        Agente.__init__(self)
        self.estado_inicial = None
        self.estado_meta = None
        self.funcion_sucesor = []
        self.tecnica = None
        self.heuristica = None
        self.memo = {}
        self.visitados = set()

    def add_funcion(self, f):
        self.funcion_sucesor.append(f)

    def test_objetivo(self, e):
        return e == self.estado_meta

    def generar_hijos(self, e):
        hijos = []
        for fun in self.funcion_sucesor:
            h = fun(e)
            hijos.append(h)
        return hijos

    def get_costo(self, camino):
        raise Exception("Error: No existe implementacion")

    def get_heuristica(self, camino):
        raise Exception("Error: No existe implementacion")

    def get_funcion_a(self, camino):
        return self.get_costo(camino) + self.get_heuristica(camino) 

    def mide_tiempo(funcion):
        def funcion_medida(*args, **kwards):
            inicio = time.time()
            c = funcion(*args, **kwards)
            t = time.time() - inicio
            return c, t
        return funcion_medida

    def estado_to_string(self, estado):
        return hashlib.md5(str(estado).encode()).hexdigest()

    @mide_tiempo
    def programa(self):
        frontera = [[self.estado_inicial]]
        contador = 0
        while frontera:
            if self.tecnica == "profundidad":
                camino = frontera.pop()
            else:
                camino = frontera.pop(0)
            nodo = camino[-1]
            nodo_str = self.estado_to_string(nodo)
            self.visitados.add(nodo_str)
            if self.test_objetivo(nodo):
                self.acciones = camino
                break
            else:
                for hijo in self.generar_hijos(nodo):
                    hijo_str = self.estado_to_string(hijo)
                    if hijo_str not in self.visitados:
                        aux = deepcopy(camino)
                        aux.append(hijo)
                        frontera.append(aux)
                if self.tecnica == "costouniforme":
                    frontera.sort(key=lambda tup: self.get_costo(tup))
                elif self.tecnica == "codicioso":
                    frontera.sort(key=lambda tup: self.get_heuristica(tup[-1]))
                elif self.tecnica == "a_estrella":
                    frontera.sort(key=lambda tup: self.get_funcion_a(tup[-1]))


class AgenteSolver(AgenteBuscador):

    def __init__(self):
        AgenteBuscador.__init__(self)
        

    def get_costo(self, camino):
        return len(camino) - 1
    
    def get_heuristica(self, camino):
        estado_str = self.estado_to_string(camino)
        if(estado_str in self.memo):
            return self.memo[estado_str]
        else:
            if(self.heuristica == "manhattan"):
                posiciones = {1: [0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1], 0:[2,2]}
                calculo = 0
                for i in range(len(camino)):
                    for j in range(len(camino[i])):
                        if(camino[i][j] != 0):
                            posicion_elemento_actual = posiciones[camino[i][j]]
                            if(posicion_elemento_actual != [i, j]):
                                calculo += (abs(posicion_elemento_actual[0] - i) + abs(posicion_elemento_actual[1] - j))
                
                self.memo[estado_str] = calculo
                return calculo
            elif(self.heuristica == "hamming"):
                counter = 0
                acumulador = 0
                for i in range(len(camino)):
                    for j in range(len(camino[0])):
                        counter+=1
                        if(camino[i][j] != counter):
                            if(camino[i][j] == 0 and counter == 9):
                                 continue
                            else:
                                acumulador+=1
                self.memo[estado_str] = acumulador
                return acumulador
            elif(self.heuristica == "ale"):
                suma_diferencias_filas = 0
                for row_1, row_2 in zip(camino, self.estado_meta):
                    suma_fila_estado_actual = sum(num for num in row_1)
                    suma_fila_estado_meta = sum(num for num in row_2)
                    suma_diferencias_filas += abs(suma_fila_estado_actual - suma_fila_estado_meta)
                self.memo[estado_str] = suma_diferencias_filas
                return suma_diferencias_filas

            elif(self.heuristica == "pitagorica"):
                posiciones = {1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1], 0: [2, 2]}
                suma_distancias = 0
            
                for i in range(len(camino)):
                    for j in range(len(camino[i])):
                        if camino[i][j] != 0:
                            posicion_elemento_actual = posiciones[camino[i][j]]
                            distancia = ((posicion_elemento_actual[0] - i) ** 2 + (posicion_elemento_actual[1] - j) ** 2) ** 0.5
                            suma_distancias += distancia
                self.memo[estado_str] = suma_distancias
                return suma_distancias

    def find_zero(self,mat):
        ans = (0,0)
        for i in range(len(mat)):
            for j in range(len(mat[i])):
                if(mat[i][j] == 0):
                    ans = (i, j)
                    break
        return ans

    def is_in_boundaries(self, zero, elem):
        return zero[0] + elem[0] >= 0 and zero[1] + elem[1] >= 0 and zero[0] + elem[0] < 3 and zero[1] + elem[1] < 3

    def make_son(self, mat, zero, elem):
        matias = deepcopy(mat)
        a,b = (zero[0] + elem[0], zero[1] + elem[1])
        aux = matias[a][b]
        matias [zero[0]][zero[1]] = aux
        matias[a][b] = 0
        return matias
    
    def generar_hijos(self, e):
        zero = self.find_zero(e)
        movs = [(0,1), (1,0), (-1,0), (0,-1)]    
        list = [self.make_son(e, zero, elem) for elem in movs if(self.is_in_boundaries(zero, elem))]
        return list


    def encontrar_movimiento(self, matriz1, matriz2):
        for i in range(3):
            for j in range(3):
                if matriz1[i][j] != matriz2[i][j]:
                    if matriz1[i][j] == 0: 
                        numero_movido = matriz2[i][j]
                        if i > 0 and matriz2[i-1][j] == 0:
                            return f"Mueve el {numero_movido} abajo"
                        elif i < 2 and matriz2[i+1][j] == 0:
                            return f"Mueve el {numero_movido} arriba"
                        elif j > 0 and matriz2[i][j-1] == 0:
                            return f"Mueve el {numero_movido} a la derecha"
                        elif j < 2 and matriz2[i][j+1] == 0:
                            return f"Mueve el {numero_movido} a la izquierda"
                    elif matriz2[i][j] == 0:
                        numero_movido = matriz1[i][j]
                        if i > 0 and matriz1[i-1][j] == 0:
                            return f"Mueve el {numero_movido} arriba"
                        elif i < 2 and matriz1[i+1][j] == 0:
                            return f"Mueve el {numero_movido} abajo"
                        elif j > 0 and matriz1[i][j-1] == 0:
                            return f"Mueve el {numero_movido} a la izquierda"
                        elif j < 2 and matriz1[i][j+1] == 0:
                            return f"Mueve el {numero_movido} a la derecha"
        return "Movimiento no identificado"
    
    def encontrar_movimiento_numeral(self, matriz1, matriz2):
        for i in range(3):
            for j in range(3):
                if matriz1[i][j] != matriz2[i][j]:
                    if matriz1[i][j] == 0: 
                        numero_movido = matriz2[i][j]
                        if i > 0 and matriz2[i-1][j] == 0:
                            return (numero_movido, 'abajo')
                        elif i < 2 and matriz2[i+1][j] == 0:
                            return (numero_movido, 'arriba')
                        elif j > 0 and matriz2[i][j-1] == 0:
                            return (numero_movido, 'derecha')
                        elif j < 2 and matriz2[i][j+1] == 0:
                            return (numero_movido, 'izquierda')
                    elif matriz2[i][j] == 0:
                        numero_movido = matriz1[i][j]
                        if i > 0 and matriz1[i-1][j] == 0:
                            return (numero_movido, 'arriba')
                        elif i < 2 and matriz1[i+1][j] == 0:
                            return (numero_movido, 'abajo')
                        elif j > 0 and matriz1[i][j-1] == 0:
                            return (numero_movido, 'izquierda')
                        elif j < 2 and matriz1[i][j+1] == 0:
                            return (numero_movido, 'derecha')
        return None
    

    
    def generar_instrucciones(self):
        instrucciones = []
        for k in range(len(self.acciones) - 1):
            instruccion = self.encontrar_movimiento(self.acciones[k], self.acciones[k + 1])
            instrucciones.append(instruccion)
        return instrucciones
    
    def decir_instruccion(self):
        instruccion = self.generar_instrucciones()[0]
        print(instruccion)
        engine.say(instruccion)
        engine.runAndWait()

class EntornoPuzzle(Entorno):
    def __init__(self):
        super().__init__()
        self.estado_inicial = None
        self.estado_meta = None
    
    def get_percepciones(self, a):
        a.estado_inicial = self.estado_inicial
        a.estado_meta = self.estado_meta
        a.programa()
    
    def ejecutar(self, agente):
        if agente.acciones:
            print("Solución encontrada:")
            for estado in agente.acciones:
                for fila in estado:
                    print(fila)
                print()
        else:
            print("No se encontró solución.")
        agente.habilitado = False 


solver = AgenteSolver()
solver.tecnica = "a_estrella"
solver.heuristica = "manhattan"

solver.estado_inicial = [[0, 2, 7], [1, 3, 8], [6, 5, 4]]
solver.estado_meta = [[1,2,3],[4,5,6],[7,8,0]]
solver.programa()

(None, 0.012007713317871094)

In [83]:
import random
import math

def es_soluble(estado):
    plano = [num for fila in estado for num in fila if num != 0]
    inversiones = sum(1 for i in range(len(plano)) for j in range(i + 1, len(plano)) if plano[i] > plano[j])
    return inversiones % 2 == 0

def generar_estado_inicial_soluble():
    while True:
        numeros = list(range(9))
        random.shuffle(numeros)
        matriz = [numeros[i:i + 3] for i in range(0, 9, 3)]
        if es_soluble(matriz):
            return matriz
        
def series_sum(factor, solution_size):
    return sum(factor**i for i in range(solution_size+1))


def effective_branching_factor(visited_nodes, solution_size, precision=0.00001):
    lower_bound = 1.0
    upper_bound = math.log(visited_nodes, solution_size)
    while upper_bound - lower_bound > precision:
        mid = (lower_bound + upper_bound) / 2
        sum = series_sum(mid, solution_size)
        if sum > visited_nodes - 1:
            upper_bound = mid
        else:
            lower_bound = mid
    return (lower_bound + upper_bound) / 2

def estados_aleatorios(cantidad):
    estados = []
    while len(estados) < cantidad:
        estado = generar_estado_inicial_soluble()
        if estado not in estados:
            estados.append(estado)
    return estados

def ejecutar_busqueda(tecnica, heuristica, estado, complejidad):
    agente = AgenteSolver()
    agente.tecnica = tecnica
    agente.heuristica = heuristica
    agente.estado_inicial = estado
    agente.estado_meta = [[1,2,3],[4,5,6],[7,8,0]]
    _, t_amp = agente.programa()
    complejidad[0].append(t_amp)
    complejidad[1].append(len(agente.visitados))
    complejidad[2].append(effective_branching_factor(len(agente.visitados), len(agente.acciones)))
    return len(agente.acciones)

def actualizar_puntos(longitud_a, longitud_b):
    if longitud_a < longitud_b:
        return 1, 0
    elif longitud_b < longitud_a:
        return 0, 1
    else:
        return 0.5, 0.5

def calcular_complejidad_algoritmos(cantidad_estados, heuristicas):
    estados = estados_aleatorios(cantidad_estados)
    resultados = {}
    
    for heuristica in heuristicas:
        resultados[f"estrella_{heuristica}"] = {"complejidad": [[], [], []], "puntos": 0}
        resultados[f"codicioso_{heuristica}"] = {"complejidad": [[], [], []], "puntos": 0}
        
        for estado in estados:
            longitud_estrella = ejecutar_busqueda("a_estrella", heuristica, estado, resultados[f"estrella_{heuristica}"]["complejidad"])
            longitud_codicioso = ejecutar_busqueda("codicioso", heuristica, estado, resultados[f"codicioso_{heuristica}"]["complejidad"])
            
            puntos_codicioso, puntos_estrella = actualizar_puntos(longitud_codicioso, longitud_estrella)
            resultados[f"codicioso_{heuristica}"]["puntos"] += puntos_codicioso
            resultados[f"estrella_{heuristica}"]["puntos"] += puntos_estrella
    
    for algoritmo, datos in resultados.items():
        print(f"Probabilidad de que {algoritmo.upper()} encuentre una solución óptima: {(datos['puntos']/cantidad_estados)*100}%")
    
    return resultados

def calcular_estadisticas(complejidad):
    tiempo_promedio = sum(complejidad[0]) / len(complejidad[0])
    nodos_promedio = sum(complejidad[1]) / len(complejidad[1])
    factor_ramificacion_promedio = sum(complejidad[2]) / len(complejidad[2])

    return {
        "Tiempo promedio": f"{tiempo_promedio:.10f} seg",
        "Nodos visitados promedio": f"{nodos_promedio:.2f}",
        "Factor de ramificación promedio": f"{factor_ramificacion_promedio:.2f}"
    }

def obtener_estadisticas(resultados):
    for algoritmo, datos in resultados.items():
        print(f"Estadísticas para {algoritmo}:")
        estadisticas = calcular_estadisticas(datos["complejidad"])
        for clave, valor in estadisticas.items():
            print(f"{clave}: {valor}")
        print()

def ejecutar_pruebas(cantidad_estados, heuristicas):
    resultados = calcular_complejidad_algoritmos(cantidad_estados, heuristicas)
    obtener_estadisticas(resultados)


cantidad_estados = 1000
heuristicas = ["manhattan", "hamming", "pitagorica"]
ejecutar_pruebas(cantidad_estados, heuristicas)

Probabilidad de que ESTRELLA_MANHATTAN encuentre una solución óptima: 50.0%
Probabilidad de que CODICIOSO_MANHATTAN encuentre una solución óptima: 50.0%
Probabilidad de que ESTRELLA_HAMMING encuentre una solución óptima: 50.0%
Probabilidad de que CODICIOSO_HAMMING encuentre una solución óptima: 50.0%
Probabilidad de que ESTRELLA_PITAGORICA encuentre una solución óptima: 50.05%
Probabilidad de que CODICIOSO_PITAGORICA encuentre una solución óptima: 49.95%
Estadísticas para estrella_manhattan:
Tiempo promedio: 0.2107300155 seg
Nodos visitados promedio: 274.57
Factor de ramificación promedio: 1.06

Estadísticas para codicioso_manhattan:
Tiempo promedio: 0.2054471631 seg
Nodos visitados promedio: 274.57
Factor de ramificación promedio: 1.06

Estadísticas para estrella_hamming:
Tiempo promedio: 1.0962200255 seg
Nodos visitados promedio: 744.90
Factor de ramificación promedio: 1.09

Estadísticas para codicioso_hamming:
Tiempo promedio: 1.0546069896 seg
Nodos visitados promedio: 744.90
Factor